In [2]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 44.1 MB/s eta 0:00:0000:01
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

#ChromeDriverManager 설치 전에 selenium쪽에서 옵션 설정하여 설치 및 실행
chrome_options = Options()
chrome_options.add_argument("--start-maximized") #브러우저 창 최대화
chrome_options.add_argument("--disable-blink-features=AutomationControlled") #자동화 감지 방지
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시줄 제거
chrome_options.add_experimental_option("useAutomationExtension", False)  # 자동화 확장 비활성화
chrome_options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

chrome_options.add_argument("--disable-dev-shm-usage") #용량이 작은(약64MB)의 /dev/shm 공유 메모리 공간을 사용하지 않게하여 많은 javascript 로드로 인한 크래시 방지

driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=chrome_options)

driver.get("https://www.coupang.com/")

print(driver)

title = driver.title
print(f"웹사이트 제목: {title}")

driver.quit()

<selenium.webdriver.chrome.webdriver.WebDriver (session="5928bc18f3d4e92d2c710e293026acd7")>
웹사이트 제목: 로켓배송으로 빠르게, 로켓와우 멤버십으로 할인과 무료 반품까지 | 쿠팡


In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # 브라우저 창을 최대화하여 모든 요소가 보이도록 함
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 감지 방지
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시줄 제거
chrome_options.add_experimental_option("useAutomationExtension", False)  # 자동화 확장 기능 비활성화
chrome_options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")  # 일반 사용자의 브라우저처럼 보이게 함
chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 2})  # 알림 차단
chrome_options.add_argument("--disable-dev-shm-usage")  # 공유 메모리 부족 문제 방지 (리눅스 환경에서 크래시 방지)
chrome_options.add_argument("--no-sandbox")  # 샌드박스 비활성화 (권한 관련 오류 방지)

try:
    # 웹드라이버 설정 - 예외 처리로 초기화 오류에 대응
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # 페이지 로드 전 짧은 대기 - 초기화 안정화를 위함
    time.sleep(1)
    
    # 쿠팡 메인 페이지 접속
    driver.get("https://www.coupang.com/")
    print("쿠팡 페이지 제목:", driver.title)
    
    # 검색창 찾기 - 명시적 대기로 페이지 로딩 기다림
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "headerSearchKeyword"))
    )
    
    # 검색어 입력 및 검색
    search_box.clear()
    search_box.send_keys("단백질 음료")
    search_box.send_keys(Keys.RETURN)
    
    # 페이지 로드 대기
    time.sleep(3)
    
    # 판매량 순으로 정렬 (판매량 순 버튼 클릭)
    try:
        sort_buttons = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.sorter-link"))
        )
        
        # 판매량 순 버튼 찾기
        for button in sort_buttons:
            if "판매량순" in button.text:
                button.click()
                print("판매량순 정렬 클릭 성공")
                break
    except Exception as e:
        print(f"정렬 버튼 클릭 중 오류 발생: {e}")
    
    # 정렬 후 페이지 로드 대기
    time.sleep(3)
    
    # 상품 정보 수집
    products = []
    
    # 첫 페이지 상품 정보 수집
    product_cards = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.search-product"))
    )
    
    print(f"총 {len(product_cards)}개의 상품을 찾았습니다.")
    
    # 상품 정보 추출
    for card in product_cards:
        try:
            # 상품명
            name_elem = card.find_element(By.CSS_SELECTOR, "div.name")
            name = name_elem.text
            
            # 가격
            price_elem = card.find_element(By.CSS_SELECTOR, "strong.price-value")
            price = price_elem.text
            
            # 평점 (없을 수도 있음)
            try:
                rating_elem = card.find_element(By.CSS_SELECTOR, "em.rating")
                rating = rating_elem.text
            except:
                rating = "평점 없음"
            
            # 리뷰 수 (없을 수도 있음)
            try:
                review_count_elem = card.find_element(By.CSS_SELECTOR, "span.rating-total-count")
                review_count = review_count_elem.text.strip("()")
            except:
                review_count = "0"
            
            # 상품 정보를 딕셔너리로 저장 (필요 없는 정보 제외)
            product_info = {
                "상품명": name,
                "가격": price,
                "평점": rating,
                "리뷰수": review_count
            }
            
            products.append(product_info)
            
        except Exception as e:
            print(f"상품 정보 추출 중 오류: {e}")
    
    # 데이터프레임 생성
    df = pd.DataFrame(products)
    
    # 현재 주피터 노트북의 경로에 엑셀 파일로 저장
    current_path = os.getcwd()
    excel_filename = os.path.join(current_path, "쿠팡_단백질음료_판매량순.xlsx")
    
    # 엑셀 파일로 저장
    df.to_excel(excel_filename, index=False)
    print(f"데이터가 {excel_filename}에 저장되었습니다.")
    
except Exception as e:
    print(f"예상치 못한 오류 발생: {e}")
    
finally:
    # 작업 완료 후 드라이버 종료 - 메모리 누수 방지
    if 'driver' in locals() and driver is not None:
        driver.quit()
        print("브라우저가 종료되었습니다.")

쿠팡 페이지 제목: 로켓배송으로 빠르게, 로켓와우 멤버십으로 할인과 무료 반품까지 | 쿠팡
정렬 버튼 클릭 중 오류 발생: Message: 

총 43개의 상품을 찾았습니다.
데이터가 /Users/jisukim/projects/00_web/crawling/쿠팡_단백질음료_판매량순.xlsx에 저장되었습니다.
브라우저가 종료되었습니다.
